In [1]:
pip install serpapi

In [12]:
import requests
import json
import time
import ast
import re

calls = 0
CALLS_PER_MIN_RATE_LIMIT = 5

def remove_first_line(test_string):
    if test_string.startswith("Here") and test_string.split("\n")[0].strip().endswith(":"):
        return re.sub(r'^.*\n', '', test_string, count=1)
    return test_string

def generate_text(prompt, model="claude-3-haiku-20240307", max_tokens=1000, temperature=0.7):
    time.sleep(16) #so it won't hit the 5 in one minute limit
    headers = {
        "x-api-key": ANTHROPIC_API_KEY,
        "anthropic-version": "2023-06-01",
        "content-type": "application/json"
    }
    data = {
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "system": "You are a world-class researcher. Analyze the given information and generate a well-structured report.",
        "messages": [{"role": "user", "content": prompt}],
    }
    response = requests.post("https://api.anthropic.com/v1/messages", headers=headers, json=data)
    print(response.json())
    response_text = response.json()['content'][0]['text']
    print(remove_first_line(response_text.strip()))
    return remove_first_line(response_text.strip())

def search_web(search_term):
    url = f"https://serpapi.com/search.json?q={search_term}&api_key={SERP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    print(data)
    return data

def generate_subtopic_report(subtopic):
    search_data = []
    all_queries = []

    print(f"Generating initial search queries for subtopic: {subtopic}...")
    initial_queries_prompt = f"Generate 3 search queries to gather information on the subtopic '{subtopic}'. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
    initial_queries = ast.literal_eval('[' + generate_text(initial_queries_prompt).split('[')[1])
    print(initial_queries)
    all_queries.extend(initial_queries)

    for i in range(2):
        print(f"Performing search round {i+1} for subtopic: {subtopic}...")
        for query in initial_queries:
            search_results = search_web(query)
            search_data.append(search_results)

        print(f"Generating additional search queries for subtopic: {subtopic}...")
        additional_queries_prompt = f"Here are the search results so far for the subtopic '{subtopic}':\n\n{str(search_data)}\n\n---\n\nHere are all the search queries you have used so far for this subtopic:\n\n{str(all_queries)}\n\n---\n\nBased on the search results and previous queries, generate 3 new and unique search queries to expand the knowledge on the subtopic '{subtopic}'. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
        additional_queries = ast.literal_eval('[' + generate_text(additional_queries_prompt).split('[')[1])
        initial_queries = additional_queries
        all_queries.extend(additional_queries)

    print(f"Generating initial report for subtopic: {subtopic}...")
    report_prompt = f"When writing your report, make it incredibly detailed, thorough, specific, and well-structured. Use Markdown for formatting. Analyze the following search data and generate a comprehensive report on the subtopic '{subtopic}':\n\n{str(search_data)}"
    report = generate_text(report_prompt, max_tokens=1000)
    """
    for i in range(3):
        print(f"Analyzing report and generating additional searches (Round {i+1}) for subtopic: {subtopic}...")
        analysis_prompt = f"Analyze the following report on the subtopic '{subtopic}' and identify areas that need more detail or further information:\n\n{report}\n\n---\n\nHere are all the search queries you have used so far for this subtopic:\n\n{str(all_queries)}\n\n---\n\nGenerate 3 new and unique search queries to fill in the gaps and provide more detail on the identified areas. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
        additional_queries = ast.literal_eval('[' + generate_text(analysis_prompt).split('[')[1])
        all_queries.extend(additional_queries)

        round_search_data = []
        for query in additional_queries:
            search_results = search_web(query)
            round_search_data.append(search_results)

        print(f"Updating report with additional information (Round {i+1}) for subtopic: {subtopic}...")
        update_prompt = f"Update the following report on the subtopic '{subtopic}' by incorporating the new information from the additional searches. Keep all existing information... only add new information:\n\n{report}\n\n---\n\nAdditional search data for this round:\n\n{str(round_search_data)}\n\n---\n\nGenerate an updated report that includes the new information and provides more detail in the identified areas. Use Markdown for formatting."
        report = generate_text(update_prompt, max_tokens=1000)

    print(f"Generating boss feedback for subtopic: {subtopic}...")
    feedback_prompt = f"Imagine you are the boss reviewing the following report on the subtopic '{subtopic}':\n\n{report}\n\n---\n\nProvide constructive feedback on what information is missing or needs further elaboration in the report. Be specific and detailed in your feedback."
    feedback = generate_text(feedback_prompt, max_tokens=1000)

    print(f"Generating final round of searches based on feedback for subtopic: {subtopic}...")
    final_queries_prompt = f"Based on the following feedback from the boss regarding the subtopic '{subtopic}':\n\n{feedback}\n\n---\n\nGenerate 3 search queries to find the missing information and address the areas that need further elaboration. Return your queries in a Python-parseable list. Return nothing but the list. Do so in one line. Start your response with [\""
    final_queries = ast.literal_eval('[' + generate_text(final_queries_prompt).split('[')[1])
    all_queries.extend(final_queries)

    final_search_data = []
    for query in final_queries:
        search_results = search_web(query)
        final_search_data.append(search_results)

    print(f"Updating report with final information for subtopic: {subtopic}...")
    final_update_prompt = f"Update the following report on the subtopic '{subtopic}' by incorporating the new information from the final round of searches based on the boss's feedback:\n\n{report}\n\n---\n\nFinal search data:\n\n{str(final_search_data)}\n\n---\n\nGenerate the final report that addresses the boss's feedback and includes the missing information. Use Markdown for formatting."
    final_report = generate_text(final_update_prompt, max_tokens=1000)

    print(f"Final report generated for subtopic: {subtopic}!")
    return final_report"""
    return report

def generate_comprehensive_report(topic, subtopic_reports):
    print("Generating comprehensive report...")
    comprehensive_report_prompt = f"Generate a comprehensive report on the topic '{topic}' by combining the following reports on various subtopics:\n\n{subtopic_reports}\n\n---\n\nEnsure that the final report is well-structured, coherent, and covers all the important aspects of the topic. Make sure that it includes EVERYTHING in each of the reports, in a better structured, more info-heavy manner. Nothing -- absolutely nothing, should be left out. If you forget to include ANYTHING from any of the previous reports, you will face the consequences. Include a table of contents. Leave nothing out. Use Markdown for formatting."
    comprehensive_report = generate_text(comprehensive_report_prompt, model="claude-3-opus-20240229", max_tokens=1000)

    print("Comprehensive report generated!")
    return comprehensive_report

# User input
research_topic = input("Enter the research topic: ")

# Generate subtopic checklist
subtopic_checklist_prompt = f"Generate a detailed checklist of subtopics to research for the topic '{research_topic}'. Return your checklist in a Python-parseable list. Return nothing but the list. Do so in one line. Maximum five sub-topics. Start your response with [\""
subtopic_checklist = ast.literal_eval('[' + generate_text(subtopic_checklist_prompt).split('[')[1])
print(f"Subtopic Checklist: {subtopic_checklist}")

# Generate reports for each subtopic
subtopic_reports = []
for subtopic in subtopic_checklist:
    subtopic_report = generate_subtopic_report(subtopic)
    subtopic_reports.append(subtopic_report)

# Combine subtopic reports into a comprehensive report
#comprehensive_report = generate_comprehensive_report(research_topic, "\n\n".join(subtopic_reports))

# Save the comprehensive report to a file
with open("comprehensive_report.txt", "a") as file:
    for report in subtopic_reports:
      file.write(report)

print("Comprehensive report saved as 'comprehensive_report.txt'.")

Enter the research topic: Isar Aerospace
{'id': 'msg_019djxukbbtZBBFAvpUnyRc6', 'type': 'message', 'role': 'assistant', 'content': [{'type': 'text', 'text': '["Company Overview", "Rocket Technology", "Launch Services", "Funding and Investments", "Market Positioning"]'}], 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 85, 'output_tokens': 28}}
["Company Overview", "Rocket Technology", "Launch Services", "Funding and Investments", "Market Positioning"]
Subtopic Checklist: ['Company Overview', 'Rocket Technology', 'Launch Services', 'Funding and Investments', 'Market Positioning']
Generating initial search queries for subtopic: Company Overview...
{'id': 'msg_01B2edcVXp7KD1rhgpPWtqRE', 'type': 'message', 'role': 'assistant', 'content': [{'type': 'text', 'text': '["company overview", "company background", "company history"]'}], 'model': 'claude-3-haiku-20240307', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'in

KeyError: 'content'